In [7]:
import torchvision.models as models
import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD
import foolbox as fb
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
from PIL import Image
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
def main() -> None:
    # instantiate a model (could also be a TensorFlow or JAX model)
    #model = models.resnet18(pretrained=True).eval()
    #model=torch.load('/data1/zyh/copycat/Framework/cifar_model.pth')
    model = nn.Sequential(
        nn.Conv2d(3, 32, 3,1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.Conv2d(32, 64, 3,1),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.Conv2d(64, 128, 3,1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Conv2d(128, 128, 3,1),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.Dropout2d(0.05),
        nn.Conv2d(128, 256, 3,1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.Conv2d(256, 256, 3,1),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.Dropout(0.1),  # type: ignore
        nn.Linear(4096, 1024),
        nn.ReLU(),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(512, 10),
    )
    path = '/data1/zyh/copycat/Framework/cifar_model.pth'
    #model.load_state_dict(torch.load('/data1/zyh/copycat/Framework/cifar_model.pth'))
    #pretrained_dict = {k: v for k, v in model_pretrained.items() if k in model_dict}
    #model_dict.update(pretrained_dict)
    #model.load_state_dict(state_dict)
    model.load_state_dict(torch.load(path),False)
    model.eval()
    print(type(model))
    preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
    fmodel = PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)


    # get data and test the model
    # wrapping the tensors with ep.astensors is optional, but it allows
    # us to work with EagerPy tensors in the following
    train_dataset = torchvision.datasets.CIFAR10(root='/data1/zyh/copycat/Framework/data',
                                             train=True,
                                             transform = Compose([Resize((256,256)),ToTensor()]),
                                             download=True)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, #该参数表示每次读取的批样本个数
                                           shuffle=True) #该参数表示读取时是否打乱样本顺序
                                           # 创建迭代器
    data_iter = iter(train_loader)

    images, labels = next(data_iter)
    # 当迭代开始时, 队列和线程开始读取数据
    #images, labels = data_iter.next()
    images=images.to(device)
    labels=labels.to(device)
    #im=images
    #images=im.resize(100,3,128,128)
    #images, labels = ep.astensors(*samples(fmodel, dataset="imagenet", batchsize=16))
    print(images.shape)
    clean_acc = accuracy(fmodel, images, labels)
    
    print(f"clean accuracy:  {clean_acc * 100:.1f} %")

    # apply the attack
    attack = LinfPGD()
    '''epsilons = [
        0.0,
        0.0002,
        0.0005,
        0.0008,
        0.001,
        0.0015,
        0.002,
        0.003,
        0.01,
        0.1,
        0.3,
        0.5,
        1.0,
    ]'''
    epsilons = [
        0.0005,
        0.001,
        0.002,
        0.01,
        0.1,
    ]
    raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
    print(type(raw_advs))
    print("atest")
    # calculate and report the robust accuracy (the accuracy of the model when
    # it is attacked)
    robust_accuracy = 1 - success.float32().mean(axis=-1)
    print("robust accuracy for perturbations with")
    for eps, acc in zip(epsilons, robust_accuracy):
        print(f"  Linf norm ≤ {eps:<6}: {acc.item() * 100:4.1f} %")

    # we can also manually check this
    # we will use the clipped advs instead of the raw advs, otherwise
    # we would need to check if the perturbation sizes are actually
    # within the specified epsilon bound
    print()
    print("we can also manually check this:")
    print()
    print("robust accuracy for perturbations with")
    for eps, advs_ in zip(epsilons, clipped_advs):
        acc2 = accuracy(fmodel, advs_, labels)
        print(f"  Linf norm ≤ {eps:<6}: {acc2 * 100:4.1f} %")
        print("    perturbation sizes:")
        perturbation_sizes = (advs_ - images).norms.linf(axis=(1, 2, 3)).numpy()
        print("    ", str(perturbation_sizes).replace("\n", "\n" + "    "))
        if acc2 == 0:
            break
    fig = plt.gcf()
    for i in range(10000):
        #plt.imshow(raw_advs[i])
        plt.imsave('./image/'+str(i) +'.png',raw_advs[i])
        i += 1
        plt.close(fig)

if __name__ == "__main__":
    main()

<class 'torch.nn.modules.container.Sequential'>
Files already downloaded and verified
torch.Size([100, 3, 32, 32])


RuntimeError: Given input size: (256x1x1). Calculated output size: (256x0x0). Output size is too small